# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words thanks to vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which is the ÌMDB dataset: it is comprised of sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-08-19 15:44:48.977026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 15:44:48.977080: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


2022-08-19 15:44:51.218198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-19 15:44:51.218270: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-19 15:44:51.218288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ASUS-Enia): /proc/driver/nvidia/version does not exist
2022-08-19 15:44:51.218507: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown here: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with word2vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with word2vec)
- the representation learned by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As in this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. The third line stores the words and their trained embeddings in `wv`.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [3]:
wv['sky']

array([-0.04877935,  0.15977468, -0.14130197,  0.23122144,  0.01369233,
       -0.31266654,  0.0406143 ,  0.47149846, -0.14982545, -0.1932919 ,
        0.08604994, -0.24297805,  0.05516991,  0.14261097, -0.00697819,
       -0.19549753,  0.16586934, -0.14413111, -0.08299018, -0.38099015,
        0.14006066,  0.01547751,  0.25288272, -0.10087724, -0.03690306,
        0.01465585, -0.15588532, -0.06743064, -0.16762848,  0.00730901,
        0.09324464,  0.02407029,  0.10750823, -0.2669721 , -0.13538092,
        0.18747288,  0.11320631, -0.06342597, -0.26093534, -0.23892604,
       -0.14263397, -0.23878504, -0.18562074,  0.15950207,  0.21733192,
       -0.03558351, -0.12860426, -0.09191859,  0.23577273,  0.22247484,
       -0.00755525, -0.15014943, -0.23986652, -0.02759789, -0.09043867,
        0.11826971,  0.17482401, -0.0467893 , -0.15115622,  0.04990044,
        0.11319101, -0.06524739,  0.04736817,  0.13338679, -0.18898356,
        0.22847487, -0.05104246,  0.20307612, -0.2714109 ,  0.19

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [4]:
len(wv['sky'])

100

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the [`Word2Vec.vw.most_similar`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words that have similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [5]:
wv.most_similar('sky', topn=10)

[('serial', 0.983917236328125),
 ('unforgettable', 0.9838810563087463),
 ('dropped', 0.9822225570678711),
 ('california', 0.9816928505897522),
 ('fascination', 0.9816314578056335),
 ('prime', 0.981250524520874),
 ('castle', 0.9808604121208191),
 ('savage', 0.9808090925216675),
 ('loggia', 0.9802732467651367),
 ('constantly', 0.9798449873924255)]

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [6]:
wv.similar_by_vector('sky', topn=10)

[('serial', 0.983917236328125),
 ('unforgettable', 0.9838810563087463),
 ('dropped', 0.9822225570678711),
 ('california', 0.9816928505897522),
 ('fascination', 0.9816314578056335),
 ('prime', 0.981250524520874),
 ('castle', 0.9808604121208191),
 ('savage', 0.9808090925216675),
 ('loggia', 0.9802732467651367),
 ('constantly', 0.9798449873924255)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [7]:
wv['good']- wv['bad']

array([-0.37166637, -0.16091965,  0.2951497 ,  0.15911925,  0.13869411,
       -0.11461031, -0.32147884, -0.19702119,  0.04642612, -0.31669348,
        0.17085645,  0.22462392,  0.10489655, -0.15274197,  0.03648162,
        0.23468664, -0.4609567 ,  0.23975718, -0.3296191 ,  0.36227012,
       -0.12528193, -0.22985701, -0.06136286, -0.04351319, -0.02858281,
       -0.14355978,  0.09355897,  0.8448734 , -0.026737  , -0.6844533 ,
       -0.6852919 ,  0.18357179, -0.11997741,  0.12232322,  0.12249207,
       -0.5228119 ,  0.33815736, -0.0301863 , -0.61012423,  0.7993257 ,
       -0.19041382, -0.2771967 , -0.36215195, -0.23147064,  0.49986258,
       -0.3300916 ,  0.32732624, -0.15710354,  0.05676132,  0.81347513,
       -0.14698789,  0.15425467, -0.4501187 , -0.08756226,  0.25061876,
        0.17150399, -0.19530614,  0.22824758, -0.00312269,  0.75429785,
        0.1330767 ,  0.28398657, -0.11979121, -0.47072142,  0.12350893,
        0.17219877, -0.2802621 ,  0.12596801,  0.5213387 ,  0.46

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [8]:
res = wv['good'] - wv['bad'] + wv['stupid']
res

array([-0.16185173, -0.18898281,  0.3933463 , -0.14649934,  0.31150448,
       -0.8036329 , -0.5329933 ,  0.37846607, -0.3781459 , -0.6257576 ,
       -0.12220848, -0.13273463,  0.18951651,  0.158674  ,  0.27098578,
       -0.02403507, -0.08583024, -0.23426285, -0.4621935 , -0.5702153 ,
        0.38977647, -0.08834568,  0.46058756, -0.24239811, -0.11323403,
       -0.33146554, -0.31188527,  0.74423534, -0.31721726, -0.63873076,
        0.01687378,  0.2791897 ,  0.14737624, -0.28553003, -0.2940798 ,
        0.25899798,  0.2724843 ,  0.04732772, -0.79719543, -0.11869627,
       -0.14768602, -0.78624046, -0.19580643,  0.2546152 ,  0.6650476 ,
       -0.23298728, -0.04553652, -0.6386329 , -0.00739868,  0.906548  ,
        0.05619894, -0.10471252, -0.51744294,  0.04136515,  0.34474483,
        0.47308308,  0.03145075,  0.23520982, -0.4643544 ,  0.40424633,
        0.14577797,  0.10626301,  0.18693808, -0.1385904 , -0.41126353,
        0.659916  , -0.14215893,  0.14354943,  0.33222494,  0.61

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `similar_by_vector` function) of `res`

In [9]:
wv.similar_by_vector(res, topn=3)

[('nice', 0.8083760738372803),
 ('given', 0.7737711668014526),
 ('used', 0.7608637809753418)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [10]:
res2 = wv['boy'] - wv['girl'] + wv['woman']

In [11]:
wv.similar_by_vector(res2, topn=3)

[('boy', 0.9572626948356628),
 ('woman', 0.9543105363845825),
 ('friend', 0.9460684061050415)]

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's check some out. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [12]:
vector_size = len(wv['sky'])
vector_size

100

❓ **Question** ❓ Use the `Word2Vec.wv.key_to_index` attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [13]:
learned_vocabulary = wv.key_to_index
learned_vocabulary

{'the': 0,
 'a': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'i': 8,
 'it': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'but': 16,
 'movie': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'one': 25,
 'be': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'they': 31,
 'an': 32,
 'so': 33,
 'like': 34,
 'who': 35,
 'from': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'if': 40,
 'out': 41,
 'about': 42,
 "it's": 43,
 'has': 44,
 'what': 45,
 'some': 46,
 'there': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'no': 52,
 'up': 53,
 'she': 54,
 'my': 55,
 'time': 56,
 'even': 57,
 'which': 58,
 'would': 59,
 'really': 60,
 'only': 61,
 'had': 62,
 'story': 63,
 'me': 64,
 'see': 65,
 'can': 66,
 'their': 67,
 'were': 68,
 'well': 69,
 'than': 70,
 'much': 71,
 'get': 72,
 'do': 73,
 'great': 74,
 'been': 75,
 'we': 76,
 'first': 77,
 'bad': 78,
 'because': 79,
 'into': 80,
 'other': 81,


In [14]:
size_learned_vocabulary = len(learned_vocabulary)
size_learned_vocabulary

8006

In [15]:
import numpy as np
unique_X_train = len(np.unique(X_train))     #esta contando apenas as frases nao as palavras nas listas !!!! FIX IT !!!

/home/eolivio/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [16]:
print(f" size of the learned vocabulary: {size_learned_vocabulary}, \n number of different words in X_train : {unique_X_train}") 

 size of the learned vocabulary: 8006, 
 number of different words in X_train : 2498


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [17]:
word2vec_3 = Word2Vec(sentences=X_train, min_count=9)
word2vec_4 = Word2Vec(sentences=X_train, min_count=1)
word2vec = Word2Vec(sentences=X_train, min_count=5)

In [18]:
word2vec = word2vec.wv

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [19]:
len(word2vec_3.wv.key_to_index) #min_count = 9

4924

In [20]:
len(word2vec_4.wv.key_to_index) #min_count = 1

30419

In [21]:
len(word2vec) #min_count = 5

8006

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

Remember that we said word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [22]:
word2vec_5 = Word2Vec(sentences=X_train, min_count=9, window=10)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [23]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    matrix = []
    for i in sentence:
        if i in word2vec.key_to_index:
            matrix.append(word2vec[i])
    return np.array(matrix)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [28]:
def embedding(word2vec, sentences):
    
    embedded_sentences =[]
    for i in sentences:
        embedded_sentences.append(embed_sentence(word2vec, i))
    return embedded_sentences
 

X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors that can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=0)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=0)


assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

NameError: name 'X_train_pad' is not defined